# Text Correction - fastText 활용한 OCR 오인식 보안 
  
[목적]
- 주민등록증, 가족관계증명서 등 행정서류에서 이름 추출하여 OCR 변환 시, 오인식으로 잘못된 정보 처리될 수 있음(e.g. 홍길동 -> 홈길동 등)
- 대한민국 성씨 기준으로 99.7%를 차지하는 111개 성을 기준으로 fastText 통해 학습하여 **사전에 등록된 단어(성)로 변환**하여 정확도 향상  
  &rarr; 2자리 성씨의 경우(제갈, 사공, 선우, 남궁, 서문, 황보 등) ROI Detection 된 글자에서 **2자리 성 대상을 알수 없어 첫번째 자리만 비교할 수 있는 제한적 적용**

[활용 기술/라이브러리]
- fastText : 페이스북에서 word2vec(구글)의 단점을 보완하면서 만들어낸 알고리즘
- 문장 속 단어들의 조합으로 워드 임베딩을 하며, 이에 따라 학습에 사용된 적이 없는 단어에 대해서도 단어 벡터를 만들 수 있음
- 형태학적 측면의 유사성 판단 가능  
  &rarr; 우리는 **의미가 아닌 형태의 유사성으로만 판단하기 때문에 fastText 활용** 

[적용 결과]
- 1글자 비교이다 보니, 정확도 낮음  
    > 깁 -> 백, 깉 -> 탁 처럼 초성+중성 비슷한 것 보다 초성+종성 비슷한게 더 score 높음



## Import Packages

In [98]:
from gensim.models.fasttext import FastText
import gensim.models.word2vec
import numpy as np
from gensim import matutils
import re
import pandas as pd
from soynlp.hangle import decompose


## fastText 모델
### 학습데이터  
[sample] 150개 성씨 대상으로 학습  
__label__1 김  
__label__1 이  
__label__1 박  

### 모델 생성 및 학습
- model = FastText(학습데이터셋, min_count, ....)
> min_count : 학습 시 지정한 숫자 보다 숫자보다 적게 발생하는 단어들은 학습하지 않음(빈도가 적은 단어들은 학습하지 않기 위함이나 우리는 문장 말뭉치가 아니여서 1로 지정)  
  word_ngrams : default word_ngrams=1 => n-gram 사용, 0 => 미사용(word2vec과 동일)  
  vector_size : 학습할 임베딩의 크기. 즉, 임베딩된 벡터의 차원  # 300이 적절  
  epchs : default epchs=5 &rarr; epoch 늘려도 성능 큰 차이 없음. 오히려 예상한 결과과 다른 단어가 우선수위화 됨  
  sg : default sg=0 => CBOW, if sg=1 => skip-gram (2가지 학습 알고리즘의 성능 차이는 크지 않음)  
  workers : 학습시 사용하는 프로세스 개수  
  window : context window 크기 # 5~10이 적절

In [99]:
corpus_path = 'name_word.txt'
model_name = 'text_correciton_name_model'

corpus = gensim.models.word2vec.Text8Corpus(corpus_path) 

#model = FastText(corpus, window=5, min_count=1, sg=1, word_ngrams=1, vector_size=300)
model = FastText(corpus, window=5, min_count=1, word_ngrams=1, vector_size = 300, epochs=25)
model.save(model_name)

print(f"학습 소요 시간 : {model.total_train_time}")
print(model.wv.vectors.shape)

학습 소요 시간 : 0.018907219005541265
(148, 300)


### 모델 선택


In [100]:
correction_model = FastText.load(model_name)

## 공통 Function
- word_sim : 단어 유사도 측정  
- jamo_word : 자모 단위로 단어를 분해하여 유사도 측정하여 성능 향상

In [101]:
# 두 단어 리스트 사이의 유사도 측정
def word_sim(word_A, word_b, model=correction_model):
    return model.wv.n_similarity(word_A, word_b)


# 단어를 자모 단위로 분해
def jamo_word(sent):
    doublespace_pattern = re.compile('\s+')

    def transform(char):
        try:
            if char == ' ':
                return char
            jamo = decompose(char)
            
            if len(jamo) == 1:
                return jamo, jamo_len
            jamo_ = ''.join(c if c != ' ' else '&' for c in jamo)
            return jamo_
        
        except Exception as e: # 마침표, 물음표 반환
            return char

    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_
  


## 단어 사전과 비교하여 유사성 검사
- **jamo 분리해서 유사성 검증하면 정확도 향상**  
- 글자수 같은 것끼리만 판단하도록 적용  
    &rarr; 실제 오류 케이스 확인하여 1~2글자는 글자수 같을 때만 체크하고 나머지는 전체 비교할지 등 보완

In [103]:
word = '깃'
dictionary = ['김','이','박','최','정','강','조','윤','장','임','한','오','서','신','권','황','안','송','전','홍','유','고','문','양','손','배','백','허','남','심','노','하','곽','성','차','주','우','구','민','류','나','진','지','엄','채','원','천','방','공','현','함','변','염','여','추','도','소','석','선','설','마','길','연','위','표','명','기','반','라','왕','금','옥','육','인','맹','제','모','탁','국','어','은','편','용','예','경','봉','사','부','가','복','태','목','형','계','피','두','감','음','빈','동','온','호','범','좌','팽','승','간','상','시','갈','단','소']
results = []

for i in range(len(dictionary)):
    
    #1자리 비교로 해당 조건 삭제
    #if len(word) == len(dictionary[i]):
    result_info = {}
    result_info['relation'] = dictionary[i]
    result_info['score'] = word_sim( jamo_word(dictionary[i]), jamo_word(word), correction_model) 
    #result_info['score'] = word_sim(dictionary[i], word, correction_model) 
    results.append(result_info)
        
results = sorted(results, key=lambda d:d['score'], reverse=True)
df = pd.DataFrame(results)
print('입력 단어:', word)
df.head(3)

입력 단어: 깃


,relation,score
0,김,0.997847
1,심,0.997590
2,길,0.997568
